In [ ]:
# ConversationBufferMemory
# 전체 저장
# 대화 내용 길수록 비효율
# 필요할 때 : text completion, to predict

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True) #챗모델이 사용하게끔 하려면 True로 구분, 아니면 False로 걍 문자열로 받기
memory.save_context({"input":"HI"},{"output":"How are you?"})
memory.load_memory_variables({})  


{'history': [HumanMessage(content='HI'), AIMessage(content='How are you?')]}

In [5]:
memory.save_context({"input": "HI"}, {"output": "How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='HI'),
  AIMessage(content='How are you?'),
  HumanMessage(content='HI'),
  AIMessage(content='How are you?')]}

In [6]:
memory.save_context({"input": "HI"}, {"output": "How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='HI'),
  AIMessage(content='How are you?'),
  HumanMessage(content='HI'),
  AIMessage(content='How are you?'),
  HumanMessage(content='HI'),
  AIMessage(content='How are you?')]}

In [7]:
# 윈도우 버퍼 메모리 : 대화 특정 부분만 저장
# ex) 최근 5개 대화 저장
# 저장 범위 임의 설정 가능
# 특정 크기 유지 가능한게 장점
# 단점: 챗봇이 최근 대화에만 집중한다는 것

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})
    
add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)


In [8]:
memory.load_memory_variables({})


{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [9]:
# ConversationSummaryMemory
# 대화를 자체적으로 요약해줌
# 매우 긴 대화에서는 그만큼 많은 토큰과 공간을 필요

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("Hi, I'm Wooyong. I'm a programmer at bank!", "Whow that is cool!")


In [10]:
add_message("And My wife is so beautiful person. I felt in love with here at first sight","I really envy you!")

In [11]:
get_history()

{'history': 'Wooyong, a programmer at a bank, introduces themselves. They mention their wife and how they fell in love with her at first sight. The AI expresses envy towards Wooyong.'}

In [12]:
# ConversationSummaryBufferMemory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 리미트에 도달시 오래된 메세지들을 요약
# 가장 최근의 상호작용들을 계속 추적
# 가장 최근 및 가장 오래 전에 주고 받은 메시지 모두 잊혀지지 않고 요약
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=50, # 메시지 토큰 수의 최댓값
    return_messages=True
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi, I'm Wooyong. I'm a programmer at bank!", "Whow that is cool!")

add_message(
    "And My wife is so beautiful person. I felt in love with here at first sight",
    "I really envy you!",
)

add_message(
    "I have a dream of being a successful person at work",
    "Goodness! You are so shiny guy~",
)

get_history()

KeyboardInterrupt: 

In [ ]:
# ConversationKGMemory
# 대화 중의 엔티티의 knowledge graph를 만들어줌
# 중요한 것들만 뽑아내는 요약본

from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(llm=llm, return_messages=True)  # 메시지 토큰 수의 최댓값


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi, I'm Wooyong. I'm a programmer at bank!", "Whow that is cool!")

memory.load_memory_variables({"input": "Who is Wooyong?"})

add_message("Wooyong loves his wife","It's so lovely!!")

memory.load_memory_variables({"input": "What does wooyong like?"})



{'history': [SystemMessage(content='On Wooyong: Wooyong is a programmer. Wooyong is at bank. Wooyong loves his wife.')]}

In [ ]:
# Memory on LLMChain

# 4o-mini 쓰시는 분들 중 혹시 NotImplementedError가 발생한다면
# llm = ChatOpenAI(
# model_name="gpt-4o-mini",
# temperature=0.1,
# tiktoken_model_name="gpt-3.5-turbo",
# )

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human : {question}
    You : 

"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Wooyong")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human : My name is Wooyong
    You : 



> Finished chain.


'Nice to meet you, Wooyong! How can I assist you today?'

In [ ]:
chain.predict(question="I live in Seoul!")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Wooyong
AI: Nice to meet you, Wooyong! How can I assist you today?
    Human : I live in Seoul!
    You : 



> Finished chain.


"That's great to hear! How can I assist you with anything related to Seoul or any other topic today?"

In [ ]:
chain.predict(question="Could you tell my friend, Jihoon good mentions? today is his birthday! ")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Wooyong
AI: Nice to meet you, Wooyong! How can I assist you today?
Human: I live in Seoul!
AI: That's great to hear! How can I assist you with anything related to Seoul or any other topic today?
    Human : Could you tell my friend, Jihoon good mentions? today is his birthday! 
    You : 



> Finished chain.


'Happy birthday to Jihoon! I hope he has a fantastic day filled with joy and celebration. He is lucky to have a thoughtful friend like you to remember his special day.'

In [ ]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    System: The human introduces themselves as Wooyong. The AI responds by saying, "Nice to meet you, Wooyong! How can I assist you today?"
Human: I live in Seoul!
AI: That's great to hear! How can I assist you with anything related to Seoul or any other topic today?
Human: Could you tell my friend, Jihoon good mentions? today is his birthday! 
AI: Happy birthday to Jihoon! I hope he has a fantastic day filled with joy and celebration. He is lucky to have a thoughtful friend like you to remember his special day.
    Human : What is my name?
    You : 



> Finished chain.


'Your name is Wooyong.'

In [ ]:
# Chat Based Memory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=120, memory_key="chat_history", return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Wooyong")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Wooyong

> Finished chain.


'Hello Wooyong! How can I assist you today?'

In [ ]:
chain.predict(question="I live in Seoul!")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Wooyong
AI: Hello Wooyong! How can I assist you today?
Human: I live in Seoul!

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?'

In [ ]:
chain.predict(question="What is your name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Wooyong
AI: Hello Wooyong! How can I assist you today?
Human: I live in Seoul!
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?
Human: What is your name?

> Finished chain.


'I am an AI assistant here to help you. You can call me Assistant or any other name you prefer. How can I assist you further, Wooyong?'

In [ ]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
System: The human introduces themselves as Wooyong.
AI: Hello Wooyong! How can I assist you today?
Human: I live in Seoul!
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or talk about regarding Seoul?
Human: What is your name?
AI: I am an AI assistant here to help you. You can call me Assistant or any other name you prefer. How can I assist you further, Wooyong?
Human: What is my name?

> Finished chain.


'Your name is Wooyong, as you mentioned earlier. How can I assist you today, Wooyong?'

In [5]:
# Chat Based Memory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=120, 
    # memory_key="chat_history", 
    # 기본적으로 메모리 키가 history이기 떄문에 모든 chat_history를 이걸로 대체가능
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


# LangChain은 input, output을 가져야 하는 Rule이 있음! 그래서 걍 argument하나 받아야함.
def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input":question},{"output": result.content})
    print(result)

invoke_chain("My name is wooyong")

content='Hello Wooyong! How can I assist you today?'


In [6]:
invoke_chain("What is my name?")

content='Your name is Wooyong.'
